**Workflow**:

---

1. Current Recharge Trending - Break into Yearly, Quarterly, Weekly Seasonality, plus Trend
2. Optimized Forecasting model - forecast for the next 7 days
3. daily update workflow
4. Final output - Recharge forecast Vs. Actual, Additives, Error accuracy.

In [12]:
#Import Packages
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import MSTL



In [13]:
#Current shape of the Dataset - Add seasonal decomposition

df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#Canvas

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())

#Weekly

act_weekly_seasonality = STL(df['y'], period=7)
act_weekly_model = act_weekly_seasonality.fit()
act_weekly_excel = pd.DataFrame({'date': df.index, 'y':act_weekly_model.observed, 'seasonal':act_weekly_model.seasonal})
act_weekly_excel.to_excel("act_weekly.xlsx")


#Monthly
act_monthly_seasonality = STL(df['y'], period=30)
act_monthly_model = act_monthly_seasonality.fit()
act_monthly_excel = pd.DataFrame({'date': df.index, 'y':act_monthly_model.observed, 'seasonal':act_monthly_model.seasonal})
act_monthly_excel.to_excel("act_weekly.xlsx")




#monthly_decompose = seasonal_decompose(df['y'], model='additive', period=30)
#weekly_decompose.plot()
#plt.show()




#Add seasonal components
#Add plot_components



0


In [14]:
#STL v Prophet Sanity Check

weekly_decompose_df = act_weekly_model.seasonal.to_frame('seasonal')
weekly_decompose_df['weekday'] = weekly_decompose_df.index.day_name()
weekly_summary = weekly_decompose_df.groupby('weekday')['seasonal'].mean()
weekly_summary.to_excel('STL_weekly_summary.xlsx')

monthly_decompose_df = act_monthly_model.seasonal.to_frame('seasonal')
monthly_decompose_df['day_of_month'] = monthly_decompose_df.index.day
monthly_summary = monthly_decompose_df.groupby('day_of_month')['seasonal'].mean()
monthly_summary.to_excel('STL_monthly_summary.xlsx')

In [20]:
#Logistic Regression with Fourier Conversions

#CA
df_ca = pd.read_excel('/content/CA Recharge.xlsx')
df_ca['ds'] = pd.to_datetime(df_ca['ds'])

m = Prophet(daily_seasonality=False, yearly_seasonality=False, seasonality_mode='additive')
m.add_seasonality(name='weekly', period=7, fourier_order=3)
m.add_seasonality(name='monthly', period=30, fourier_order=5)
m.add_seasonality(name='annual', period=365, fourier_order=10)
m.add_country_holidays(country_name='PAK')

m.fit(df_ca)

df_ca_future = m.make_future_dataframe(periods=7)
forecast_ca = m.predict(df_ca_future)

forecast_ca.to_excel('forecast_ca.xlsx')

weekly_decompose_df2 = forecast_ca[['ds','yhat','weekly']].copy()
weekly_decompose_df2['weekday'] = weekly_decompose_df2['ds'].dt.day_name()
weekly_summary = weekly_decompose_df2.groupby('weekday')['weekly'].mean()
weekly_summary.to_excel('FB_weekly_summary.xlsx')




INFO:prophet:Found custom seasonality named 'weekly', disabling built-in 'weekly' seasonality.


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '540 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

In [ ]:
from google.colab import drive

forecast.to_excel('/content/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [ ]:
%cd /content
!rm -rf glowing-adventure

/content


In [ ]:
#MSTL
df = pd.read_excel('/content/Ufone.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#Canvas

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())

#Model instantiation

mstl = MSTL(df['y'], periods=[7], stl_kwargs = {'seasonal':7})
mstl_re = mstl.fit()
mstl_df = pd.DataFrame(mstl_re.seasonal, index = df.index, columns=['weekly'])

weekly_seasonality = mstl_df['weekly'].to_frame('seasonality_index')
weekly_seasonality['weekday'] = weekly_seasonality.index.day_name()
#weekly_seasonality = weekly_seasonality.dropna()

weekly_summary = weekly_seasonality.groupby('weekday')['seasonality_index'].mean()
print(weekly_seasonality.isna().sum())







0
seasonality_index    1023
weekday                 0
dtype: int64
